# **Quick Quality Check**

This notebook performs a quick quality check on the Reddit data:
- Data distributions
- Sample review
- Basic statistics

[Next: Exploratory Data Analysis →](02_eda_weaklabels.ipynb)